In [130]:
import dataretrieval.nwis as nwis
import pandas as pd
import sqlite3
import numpy as np

In [2]:
conn = sqlite3.connect('purewaters.db')
c = conn.cursor()

In [172]:
mt = pd.read_excel('mt usgs.ods', engine='odf')
mt['site'] = [str(i).zfill(8) for i in mt['site']]
mt.head(5)

,site,description
0,05014500,Swiftcurrent Creek at Many Glacier MT
1,05015500,Lake Sherburne at Sherburne MT
2,05017500,St. Mary River near Babb MT
3,05018000,St. Mary Canal at intake near Babb MT
4,05018500,St. Mary Canal at St. Mary Crossing near Babb MT


In [173]:
def clean(site):
    site = str(site)
    siteInfo = nwis.get_info(sites=site)
    a = siteInfo[0]
    a = a[['site_no', 'station_nm', 'dec_lat_va', 'dec_long_va', 'alt_va']]
    a = a.rename(columns = {'site_no': 'site', 'station_nm': 'station', 'dec_lat_va':'lat', 'dec_long_va':'long',
                        'alt_va':'alt'})
    return(a)
    

In [175]:
df = pd.DataFrame(columns = ['site', 'station', 'lat', 'long', 'alt'])
lst = []
for i in mt['site']:
    a = clean(i)
    lst.append(a)
df = df.append(lst)
df

/tmp/ipykernel_3681001/3402706273.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(lst)


,site,station,lat,long,alt
0,05014500,Swiftcurrent Creek at Many Glacier MT,48.798833,-113.656731,4876.78
0,05015500,Lake Sherburne at Sherburne MT,48.828094,-113.521983,4709.45
0,05017500,St. Mary River near Babb MT,48.833037,-113.420653,4468.13
0,05018000,St. Mary Canal at intake near Babb MT,48.852778,-113.416944,4470
0,05018500,St. Mary Canal at St. Mary Crossing near Babb MT,48.946967,-113.375331,4450
...,...,...,...,...,...
0,12381400,South Fork Jocko River near Arlee MT,47.19555,-113.850736,3970
0,12388700,Flathead River at Perma MT,47.366792,-114.585,2469.3
0,12389000,Clark Fork near Plains MT,47.4292,-114.856533,2449.11
0,12389500,Thompson River near Thompson Falls MT,47.591858,-115.229536,2429.97


In [176]:
df.to_sql('usgs', conn, if_exists='replace', index=False)

244

In [177]:
pd.read_sql('select * from usgs', conn)

,site,station,lat,long,alt
0,05014500,Swiftcurrent Creek at Many Glacier MT,48.798833,-113.656731,4876.78
1,05015500,Lake Sherburne at Sherburne MT,48.828094,-113.521983,4709.45
2,05017500,St. Mary River near Babb MT,48.833037,-113.420653,4468.13
3,05018000,St. Mary Canal at intake near Babb MT,48.852778,-113.416944,4470
4,05018500,St. Mary Canal at St. Mary Crossing near Babb MT,48.946967,-113.375331,4450
...,...,...,...,...,...
239,12381400,South Fork Jocko River near Arlee MT,47.195550,-113.850736,3970
240,12388700,Flathead River at Perma MT,47.366792,-114.585000,2469.3
241,12389000,Clark Fork near Plains MT,47.429200,-114.856533,2449.11
242,12389500,Thompson River near Thompson Falls MT,47.591858,-115.229536,2429.97


In [178]:
pd.read_sql('select * from usgs where alt < 2000', conn)

,site,station,lat,long,alt
0,06175510,Missouri R at E Frazer Pump Plant nr Frazer MT,48.033351,-106.000572,1950
1,06177000,Missouri River near Wolf Point MT,48.067325,-105.533097,1958.57
2,06181000,Poplar River near Poplar MT,48.117917,-105.192817,1943
3,06185110,Big Muddy Creek nr mouth nr Culbertson MT,48.164481,-104.629519,1896.5
4,06185500,Missouri River near Culbertson MT,48.123500,-104.473300,1883.4
5,06329500,Yellowstone River near Sidney MT,47.677414,-104.155411,1881.3
6,06329610,"YELLOWSTONE R NO. 2 NR CARTWRIGHT, ND",47.861964,-103.966874,1802.64
7,12304500,Yaak River near Troy MT,48.561722,-115.970158,1839
8,12305000,"KOOTENAI RIVER AT LEONIA, ID",48.617500,-116.046389,1794.12


In [180]:
pd.read_sql('select * from usgs where alt >4500', conn)

,site,station,lat,long,alt
0,05014500,Swiftcurrent Creek at Many Glacier MT,48.798833,-113.656731,4876.78
1,05015500,Lake Sherburne at Sherburne MT,48.828094,-113.521983,4709.45
2,06006000,"Red Rock Cr ab Lakes, nr Lakeview, MT",44.615792,-111.625669,6673
3,06012500,Red Rock R bl Lima Reservoir nr Monida MT,44.655881,-112.371222,6530
4,06016000,Beaverhead River at Barretts MT,45.116128,-112.750494,5268.17
...,...,...,...,...,...
73,12329500,Flint Creek at Maxville MT,46.463758,-113.240278,4828.38
74,12330000,Boulder Creek at Maxville MT,46.471653,-113.235769,4750
75,12332000,Middle Fork Rock Cr nr Philipsburg MT,46.184569,-113.501569,5444.08
76,12335500,"Nevada Cr ab reservoir, nr Helmville, MT",46.777778,-112.767786,4640
